In [1]:
import glob
from torch.utils.data import Dataset, DataLoader
import torch
import os
from PIL import Image
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
from torchvision import  models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [2]:
import math
import torch.nn.functional as F

In [3]:
class_names="b1,b2,b3,g1,g2,g3,g4,g5".split(",")
class_names

['b1', 'b2', 'b3', 'g1', 'g2', 'g3', 'g4', 'g5']

In [4]:
spath=r"/kaggle/input/picforkaggleme"

files=glob.glob(os.path.join(spath,"*/*.*"))
files=[x for x in files if "_b" in x or "_g" in x]
files=[x for x in files if not "min" in x ]
files=[x for x in files if "_day" in x ]
files[:11]

['/kaggle/input/picforkaggleme/picforkaggle/002050_b2_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/601225_b1_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/603958_b2_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/600237_b3_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/301058_g4_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/603269_g4_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/300600_g2_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/603808_b3_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/301006_g1_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/002472_g1_day.jpg',
 '/kaggle/input/picforkaggleme/picforkaggle/002736_b3_day.jpg']

In [5]:
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

In [6]:
def collect_dictionary(obj):
  inv_obj = {}
  for key, value in obj.items():
    inv_obj.setdefault(value, list()).append(key)
  return inv_obj

In [7]:
# lbl2num={"b1":1,"b2":2,"b3":3,"g1":4,"g2":5,"g3":6,"g4":7,"g5":8}
lbl2num={"b1":0,"b2":1,"b3":2,"g1":3,"g2":4,"g3":5,"g4":6,"g5":7}
numtolbl=collect_dictionary(lbl2num)
numtolbl,lbl2num

({0: ['b1'],
  1: ['b2'],
  2: ['b3'],
  3: ['g1'],
  4: ['g2'],
  5: ['g3'],
  6: ['g4'],
  7: ['g5']},
 {'b1': 0, 'b2': 1, 'b3': 2, 'g1': 3, 'g2': 4, 'g3': 5, 'g4': 6, 'g5': 7})

In [8]:
lbl2num["b1"]

0

In [9]:
class MyDataset(Dataset):
    def __init__(self,spathpic,transform=None):
        self.x = self.donepic_(spathpic)
        self.y = self.donepic_lbl(self.x)
        self.len = len(self.x)
        self.transform = transform
        
    def __getitem__(self, index):
        img=self.x[index]## picture filename
        img = Image.open(img).convert('RGB')     # 0~255
        if self.transform is not None:
            img = self.transform(img) 
            
        return img, lbl2num[self.y[index]]

    def __len__(self):
        return self.len

    def donepic_(self,spathpic):
        files=glob.glob(os.path.join(spath,"*/*.*"))
        files=[x for x in files if "_b" in x or "_g" in x]
        files=[x for x in files if "_day" in x ]
        return files

    def donepic_lbl(self,xlist):
        lbllist=[x.split("_")[1] for x in xlist]
        return lbllist

In [10]:
ds_=MyDataset(spath,transform=train_transform)

train_ds = DataLoader(dataset=ds_, batch_size=64, shuffle=False)
train_ds

In [11]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.figure(figsize=(122, 122))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[x for x in classes])
#     imshow(out, title=[numtolbl[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(train_ds))

In [13]:
classes

tensor([1, 0, 1, 2, 6, 6, 4, 2, 3, 3, 2, 3, 1, 7, 2, 6, 4, 5, 1, 0, 7, 5, 3, 4,
        7, 2, 3, 2, 6, 1, 1, 4, 4, 3, 1, 4, 2, 6, 4, 2, 3, 1, 3, 4, 1, 0, 5, 6,
        0, 5, 4, 5, 4, 0, 2, 5, 1, 3, 3, 6, 2, 1, 3, 5])